Project Details and Goals

Build a molecular dynamics simulation to study a solid-solid phase transition. Specifically, I am looking at the phase transition of pure metallic microstructures. I am specifically look at the phase transition of alpha-iron (BCC) to gamma-iron (FCC). This is done by applying a thermostat and study the qualitative structure changes over time. Need to use a visualization tool to physicaly see the simulation.


PowerPoint Outline
- Intro: Talk about iron - phase diagram
- Applications of ferrite and austenite - properties - essentially why do these different phases exist
- Crystal structures - Explain FCC and BCC - possible order parameters to explain the crystallinity
- Methodology
- What kind of particles - LJ? Embedded Atom Model?
- JAXMD/HOO-MD/LAAMPS

-Code Overview
- Figure Out what ensemble to use (NPT vs NVT)


Live Simulation
- Am I able to do a quick live simulation using LJ particles or a different use case with nonmetal cases. Essentially I want to show the phase transition LIVE


Goals
- Exploration of MD tools, JAX-MD, LAMMPS, HOO-MD - figure out how to use the tools specificlaly for our MD simulation
- Initializing a crystal structure
- Apply a theromstat and visualize a phase transition using a visualization tool

# Importing Libraries


In [ ]:
%pip install hoomd #Using HOOMD for simulations
import numpy as np # For now we will use numpy 
import hoomd
import hoomd.md as md

# Initializing Ferrite Lattice (BCC)

Find parameters for Iron BCC structure

In [ ]:
#First gonna do BCC lattice, but may transition to BCC or FCC later

a = 2.86

def generate_bcc_lattice(a=2.86, nx=5, ny=5, nz=5): #n-dimensions represents the number of unit cells
    """Generate a BCC latice"""
    unit_cell = np.array([[0, 0, 0], [0.5, 0.5, 0.5]]) * a
    lattice = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                for pos in unit_cell:
                    lattice.append(pos + a * np.array([i, j, k]))
    return np.array(lattice)

# Uses numpy for now, just want to see if it works then transition to jax for faster initial configuration

# HOOMD System Initialization

In [ ]:
#Use CPU first then transition to GPU

# Lets HOOMD use CPU or GPU (depends on what you specify)
hoomd.context.initialize("--mode=cpu") #when I switch to GPU, use "--mode=gpu"

#Generate positions for BCC structure
n = 5 # Number of unit cells

positions = generate_bcc_lattice(a=2.86, nx=n, ny=n, nz=n)

#Create simulation box
box_length = a * n

frame = hoomd.data.make_snapshot(N=len(positions), box=hoomd.data.boxdim(Lx=box_length, Ly=box_length, Lz=box_length), particle_types=['Fe'])

frame.partcles.position[:] = positions
system = hoomd.init.read_snapshot(frame)


# EAM Potential Setup

https://hoomd-blue.readthedocs.io/en/v2.9.5/module-metal-pair.html - how to access EAM potential for Iron
https://www.ctcms.nist.gov/potentials/system/Fe/#Fe - EAM potential data here

Must add the file to the working directory containing information about the EAM paramaters for alpha-iron specifically

In [ ]:
# EAM Potential Setup

#Define neighbor list
nlist = md.nlist.cell()

#Create pair potential
eam = hoomd.metal.pair.eam(file = 'Placeholder_Fefile', type='Alloy', nlist=nlist)
eam.par_coeff.set('Fe', 'Fe')

#Make sure I reaplce the filename once that is acquired





# NPT Ensemble Setup


In [ ]:
#NPT Ensemble Setup

npt = md.integrate.npt(group=hoomd.group.all(), 
                       kT=0.5,  # initial temperature in reduced units
                       tau=1.0, 
                       P=1.0,  # atmospheric pressure
                       tauP=10.0)


#Integrator
md.integrate.mode_standard(dt=0.005)
#Explain further what the integrator does


#Outpur file for OVITO
hoomd.dump.gsd("ferrite_to_austenite.gsd", period=1000, group=hoomd.group.all(), overwrite=True)


In [ ]:
# Run Simulate with Gradually Increasing Temperature Ramp

# Run at initial low temp (simulate ferrite equilibrium)
hoomd.run(1_000_0)

# Ramp temperature up to drive phase transition
npt.set_params(kT=1.0)
hoomd.run(10_000)

npt.set_params(kT=1.5)
hoomd.run(10_000)

npt.set_params(kT=2.0)
hoomd.run(10_000)